In [ ]:
import os
import cv2
from skimage.feature import daisy
from scipy.spatial import distance
import numpy as np
from scipy import stats
from google.colab.patches import cv2_imshow

Read Images From Train

In [ ]:
descs_train = []
label_train = []
label = 0
label_names = []
root_folder = "drive/MyDrive/Deep_Learning/HW1/train/"
for foldername in os.listdir(root_folder):
  print("Folders:", foldername)
  for filename in os.listdir(root_folder + foldername):
    if filename.endswith("jpg"): 
        image = cv2.imread(os.path.join(root_folder,foldername,filename),0)
        if image is not None:
                descs = daisy(image, step=180, radius=58, rings=2, histograms=256, orientations=8, visualize=False)
                descs = np.ravel(descs)
                descs_train.append(descs[:4104])
                label_train.append(label)
  label += 1
  label_names.append(foldername)
# Check lengths are matched.  
print("Number of images for train: ", len(label_train))
print("Labels: ", label_names)

Folders: laptop
Folders: chair
Folders: butterfly
Number of images for train:  225
Labels:  ['laptop', 'chair', 'butterfly']


Read Images From Test

In [ ]:
test_images = []
descs_test = []
file_names = []
root_folder = "drive/MyDrive/Deep_Learning/HW1/test/"
for filename in os.listdir(root_folder):
    if filename.endswith("jpg"): 
        image = cv2.imread(os.path.join(root_folder,filename),0)
        file_names.append(filename)
        if image is not None:
                test_images.append(image)
                descs = daisy(image, step=180, radius=58, rings=2, histograms=256, orientations=8, visualize=False)
                descs = np.ravel(descs)
                descs_test.append(descs[:4104])

Ready to use kNN Function

In [ ]:
"""
def KNN(x_train, y_train, sample_test, k):
  from sklearn.neighbors import KNeighborsClassifier
  classifier = KNeighborsClassifier(n_neighbors=k, metric="minkowski", p=3)
  classifier.fit(x_train, y_train)
  y_pred = classifier.predict(sample_test)
  #print(y_pred)
  return y_pred
"""

'\ndef KNN(x_train, y_train, sample_test, k):\n  from sklearn.neighbors import KNeighborsClassifier\n  classifier = KNeighborsClassifier(n_neighbors=k, metric="minkowski", p=3)\n  classifier.fit(x_train, y_train)\n  y_pred = classifier.predict(sample_test)\n  #print(y_pred)\n  return y_pred\n'

My Own kNN Function for Minkowski Distance

In [ ]:
def KNN(x_train, y_train, sample_test, k):
  dist_list = []
  result = []
  for i in range(len(x_train)):
    dist = distance.minkowski(x_train[i], sample_test, p=3)
    dist_list.append(dist)
  index_sorted = np.argsort(dist_list)[:k]
  for i in index_sorted:
    result.append(y_train[i])
  print("7 Nearest Neighbour Classes:",result)
  return stats.mode(result)[0]

In [ ]:
classes = []
for i in range(len(descs_test)):
  result_class = KNN(descs_train, label_train, descs_test[i], 7)
  classes.append(label_names[int(result_class)])
for i in range(len(file_names)):
  print("{}:{}".format(file_names[i], classes[i]))
print("Result Classes of Test Images:", classes)

7 Nearest Neighbour Classes: [0, 1, 2, 2, 0, 2, 2]
7 Nearest Neighbour Classes: [2, 2, 2, 1, 2, 2, 2]
7 Nearest Neighbour Classes: [2, 2, 2, 1, 1, 2, 1]
7 Nearest Neighbour Classes: [2, 2, 0, 2, 1, 2, 2]
7 Nearest Neighbour Classes: [1, 0, 0, 1, 0, 0, 1]
7 Nearest Neighbour Classes: [2, 1, 2, 2, 1, 1, 0]
7 Nearest Neighbour Classes: [0, 0, 0, 0, 0, 0, 0]
7 Nearest Neighbour Classes: [0, 0, 0, 2, 1, 1, 0]
7 Nearest Neighbour Classes: [0, 1, 1, 0, 0, 0, 0]
image_0013.jpg:butterfly
image_0010.jpg:butterfly
image_0031.jpg:butterfly
image_0008.jpg:butterfly
image_0014.jpg:laptop
image_0001.jpg:chair
image_0007.jpg:laptop
image_0073.jpg:laptop
image_0074.jpg:laptop
Result Classes of Test Images: ['butterfly', 'butterfly', 'butterfly', 'butterfly', 'laptop', 'chair', 'laptop', 'laptop', 'laptop']


My Own kNN Function for Mahalanobis Distance


I choose descriptor lenghts as 1000. Because calculation of mahalanobis distance takes a lot of time. Therefore, I couldn't use length as 4104. 

In [ ]:
def KNN(x_train, y_train, sample_test, k):
  dist_list = []
  result = []
  for i in range(len(x_train)):
    cov_matrix = np.cov(np.array([x_train[i][:1000], sample_test[:1000]]).T)
    try:
      dist = distance.mahalanobis(x_train[i][:1000], sample_test[:1000], np.linalg.inv(cov_matrix))
    except:
      dist = distance.mahalanobis(x_train[i][:1000], sample_test[:1000], np.linalg.pinv(cov_matrix))
    dist_list.append(dist)
  index_sorted = np.argsort(dist_list)[:k]
  for i in index_sorted:
    result.append(y_train[i])
  print("7 Nearest Neighbour Classes:",result)
  return stats.mode(result)[0]

In [ ]:
classes = []
for i in range(len(descs_test)):
  result_class = KNN(descs_train, label_train, descs_test[i], 7)
  classes.append(label_names[int(result_class)])
for i in range(len(file_names)):
  print("{}:{}".format(file_names[i], classes[i]))
print("Result Classes of Test Images:", classes)

7 Nearest Neighbour Classes: [0, 2, 2, 2, 2, 1, 2]
7 Nearest Neighbour Classes: [1, 0, 0, 0, 2, 2, 0]
7 Nearest Neighbour Classes: [2, 0, 2, 2, 2, 0, 1]
7 Nearest Neighbour Classes: [0, 2, 2, 1, 0, 2, 1]
7 Nearest Neighbour Classes: [2, 0, 0, 2, 0, 0, 0]
7 Nearest Neighbour Classes: [1, 0, 0, 1, 1, 1, 2]
7 Nearest Neighbour Classes: [0, 0, 0, 0, 0, 1, 2]
7 Nearest Neighbour Classes: [0, 0, 2, 2, 2, 0, 0]
7 Nearest Neighbour Classes: [2, 2, 1, 0, 1, 2, 2]
image_0013.jpg:butterfly
image_0010.jpg:laptop
image_0031.jpg:butterfly
image_0008.jpg:butterfly
image_0014.jpg:laptop
image_0001.jpg:chair
image_0007.jpg:laptop
image_0073.jpg:laptop
image_0074.jpg:butterfly
Result Classes of Test Images: ['butterfly', 'laptop', 'butterfly', 'butterfly', 'laptop', 'chair', 'laptop', 'laptop', 'butterfly']
